In [1]:
import pandas as pd
import numpy as np
import sys
import os
import random
from math import sqrt
sys.path.append('../..')
from modules import utils
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Input, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline

2023-08-15 12:30:40.268445: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-15 12:30:40.328671: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-15 12:30:40.330631: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-15 12:30:40.330641: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
tf.random.set_seed(SEED)

#### The data

In [3]:
kampala_devices = pd.read_csv('../data/kampala_devices.csv', usecols=['lat', 'long', 'id'])
kampala_devices.head()

,id,lat,long
0,930434,0.360209,32.610756
1,718028,0.307500,32.620600
2,912224,0.346460,32.703280
3,930426,0.365500,32.646800
4,930427,0.268900,32.588000


In [4]:
kampala_df = pd.read_csv('../data/kampala_data.csv', parse_dates=['timestamp'])
kampala_df.head()

,site_name,latitude,longitude,city,timestamp,pm2_5_calibrated_value,pm2_5_raw_value,pm10_raw_value,pm10_calibrated_value,site_id,device_number,device_name
0,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 00:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
1,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 01:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
2,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 02:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
3,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 03:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
4,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 04:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26


In [5]:
final_df = pd.DataFrame()
# device_ids = []
cols = ['timestamp', 'latitude', 'longitude', 'pm2_5_calibrated_value', 'device_number']
# for i, device_id in enumerate(device_ids):
for i, device_id in kampala_devices.id.iteritems():
    device_df = utils.get_device_data(kampala_df, device_id, cols)
#     if len(device_df)!=0:
#         device_ids.append(device_id)
    processed_df = utils.preprocessing(device_df)
#     print(f'{device_id}: {len(processed_df)}')
    final_df = pd.concat([final_df, processed_df])
final_df.reset_index(drop=True, inplace=True)
final_df.head()

,time,latitude,longitude,pm2_5,device_number
0,453031.0,0.356989,32.613888,10.5477,930434
1,453032.0,0.356989,32.613888,16.4250,930434
2,453033.0,0.356989,32.613888,17.7239,930434
3,453034.0,0.356989,32.613888,16.1533,930434
4,453035.0,0.356989,32.613888,18.0123,930434


In [6]:
latitudes = final_df['latitude'].unique()
longitudes = final_df['longitude'].unique()
device_ids = final_df['device_number'].unique()
len(latitudes), len(longitudes), len(device_ids)

(35, 35, 34)

In [7]:
final_df = final_df.drop(['device_number'], axis=1)
final_df.head()

,time,latitude,longitude,pm2_5
0,453031.0,0.356989,32.613888,10.5477
1,453032.0,0.356989,32.613888,16.4250
2,453033.0,0.356989,32.613888,17.7239
3,453034.0,0.356989,32.613888,16.1533
4,453035.0,0.356989,32.613888,18.0123


#### Model training and validation

In [8]:
def ffnn(X_train, y_train, epochs=1000, optimizer='RMSProp', dropout=0.2):
    model = tf.keras.Sequential() 
    model.add(Input(shape=(X_train.shape[1],), name='Input-Layer')) 
    model.add(Dropout(rate=dropout))
    model.add(Dense(128, activation='relu', name='Hidden-Layer1'))
    model.add(Dropout(rate=dropout))
    model.add(Dense(32, activation='relu', name='Hidden-Layer2'))
    model.add(Dropout(rate=dropout))
    model.add(Dense(1, activation='linear', name='Output-Layer')) 

    model.compile(optimizer=optimizer, # default='rmsprop', an algorithm to be used in backpropagation
                  loss=tf.keras.losses.MeanSquaredError(),
                 )
    checkpoint = ModelCheckpoint('../models/ffnn_checkpoint.h5', monitor='val_loss', svae_best_only=True,
                                save_weights_only=False)
    early_stopping = EarlyStopping(monitor='val_loss', patience=300)
    model.fit(X_train, 
          y_train,
          batch_size=32,
          epochs=epochs, # default=1, Number of epochs to train the model
          callbacks=[checkpoint, early_stopping],
          validation_split=0.2, 
         )
    best_model = keras.models.load_model('../models/ffnn_checkpoint.h5')
    return best_model

In [9]:
# def scale_data(X):
#     scaler = MinMaxScaler()
#     X_scaled = X.copy()
#     X_scaled[:, 0] = scaler.fit_transform(X[:, 0].reshape(-1, 1)).flatten()
#     return X_scaled

#### delete from here

In [10]:
# idx = 0
# device_indices = final_df[final_df.latitude==latitudes[idx]].index
# device_df = jinja_df[jinja_df.device_number == device_ids[idx]]
# assert(len(device_indices) == len(device_df)-device_df.pm2_5_calibrated_value.isna().sum())
# test_df = final_df.loc[device_indices]
# test_df.head()

In [11]:
# train_df = pd.concat([final_df, test_df]).drop_duplicates(keep=False)
# assert(len(train_df.longitude.unique()) == len(longitudes)-1)
# assert len(final_df) == len(test_df) + len(train_df)
# train_df.head()

In [12]:
# X_train = train_df.iloc[:, 0:-1]
# y_train = train_df.iloc[:, -1]
# X_train, y_train = np.array(X_train), np.array(y_train)#.reshape(-1, 1)

# X_test = test_df.iloc[:, 0:-1]
# y_test = test_df.iloc[:, -1]
# X_test, y_test = np.array(X_test), np.array(y_test)#.reshape(-1, 1)
# X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [13]:
# X_train[0]

In [14]:
# X_train_scaled = scale_data(X_train)
# X_train_scaled

In [15]:
# model = ffnn(X_train, y_train, 500, 'RMSProp')

In [16]:
# y_pred = model.predict(X_test) 
# y_pred.shape, y_test.reshape(-1, 1).shape

In [17]:
# np.min(y_pred), np.mean(y_pred), np.max(y_pred)

In [18]:
# mean_squared_error(y_test, y_pred)

In [19]:
# y_train_pred = model.predict(X_train)
# mean_squared_error(y_train, y_train_pred)

#### end here

In [20]:
def cross_validation(final_df, idx):
    device_indices = final_df[final_df.latitude==latitudes[idx]].index
    device_df = kampala_df[kampala_df.device_number == device_ids[idx]]
#     assert(len(device_indices) == len(device_df)-device_df.pm2_5_calibrated_value.isna().sum())
    
    test_df = final_df.loc[device_indices]
#     assert(len(test_df.longitude.unique()) == 1)
    
    train_df = pd.concat([final_df, test_df]).drop_duplicates(keep=False)
#     assert(len(train_df.longitude.unique()) == len(longitudes)-1)
#     assert len(final_df) == len(test_df) + len(train_df)
    
    X_train = train_df.iloc[:, 0:-1]
    y_train = train_df.iloc[:, -1]
    X_train, y_train = np.array(X_train), np.array(y_train)#.reshape(-1, 1)
#     X_train_scaled = scale_data(X_train)
    
    X_test = test_df.iloc[:, 0:-1]
    y_test = test_df.iloc[:, -1]
    X_test, y_test = np.array(X_test), np.array(y_test)#.reshape(-1, 1)
#     X_test_scaled = scale_data(X_test)
    
    model = ffnn(X_train, y_train)#, optimizer='Adam', dropout=0.8)
    y_pred = model.predict(X_test) 
    
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    return rmse, mape

In [21]:
latitudes

array([0.35698941, 0.31197891, 0.35068083, 0.3614801 , 0.27305058,
       0.34483917, 0.32907575, 0.3238265 , 0.37307176, 0.3841366 ,
       0.31079195, 0.36384297, 0.37791733, 0.30562933, 0.21854908,
       0.27221229, 0.35192674, 0.29776972, 0.35135358, 0.3024902 ,
       0.34025631, 0.33906876, 0.28461732, 0.33555934, 0.29082243,
       0.35553887, 0.35421934, 0.31772521, 0.06349111, 0.23247482,
       0.28349049, 0.39542619, 0.32005559, 0.29030967, 0.30746479])

In [22]:
latitudes[:10]

array([0.35698941, 0.31197891, 0.35068083, 0.3614801 , 0.27305058,
       0.34483917, 0.32907575, 0.3238265 , 0.37307176, 0.3841366 ])

In [23]:
rmse_list, mape_list = [], []
# for i in range(len(latitudes[30:])):
lat_list = list(latitudes)
for lat in latitudes[30:]:
    i = lat_list.index(lat)
#     print(i)
    try:
        rmse, mape = cross_validation(final_df, i)
        rmse_list.append(rmse)
        mape_list.append(mape)
        print(f'Location {i} successful')
    except Exception as e:
        print(e)
        print(f'Location {i} failed')
#         break

mean_rmse = np.mean(rmse_list)
mean_mape = np.mean(mape_list)
mean_rmse, mean_mape

Epoch 1/1000


2023-08-15 12:31:32.396656: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-08-15 12:31:32.396683: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: PL1207-PRO.paris.inria.fr
2023-08-15 12:31:32.396686: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: PL1207-PRO.paris.inria.fr
2023-08-15 12:31:32.396780: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 525.125.6
2023-08-15 12:31:32.396801: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: NOT_FOUND: could not find kernel module information in driver version file contents: "NVRM version: NVIDIA UNIX Open Kernel Module for x86_64  525.125.06  Release Build  (dvs-builder@U16-A23-14-1)  Tue May 30 05:12:48 UTC 2023
GCC version:  gcc ve

1167/1167 [==============================] - 1s 1ms/step - loss: 28124340.0000 - val_loss: 1476.6328
Epoch 2/1000
1167/1167 [==============================] - 1s 952us/step - loss: 5333.6904 - val_loss: 1679.7161
Epoch 3/1000
1167/1167 [==============================] - 1s 966us/step - loss: 2934.8176 - val_loss: 998.4408
Epoch 4/1000
1167/1167 [==============================] - 1s 970us/step - loss: 3091.9563 - val_loss: 794.5401
Epoch 5/1000
1167/1167 [==============================] - 1s 961us/step - loss: 2763.7495 - val_loss: 789.7619
Epoch 6/1000
1167/1167 [==============================] - 1s 939us/step - loss: 1148.5487 - val_loss: 758.4896
Epoch 7/1000
1167/1167 [==============================] - 1s 901us/step - loss: 826.5466 - val_loss: 716.9061
Epoch 8/1000
1167/1167 [==============================] - 1s 914us/step - loss: 812.1600 - val_loss: 731.9813
Epoch 9/1000
1167/1167 [==============================] - 1s 967us/step - loss: 729.1613 - val_loss: 733.8348
Epoch 10/1000

1167/1167 [==============================] - 1s 937us/step - loss: 531.9858 - val_loss: 623.5977
Epoch 76/1000
1167/1167 [==============================] - 1s 995us/step - loss: 532.3965 - val_loss: 623.3426
Epoch 77/1000
1167/1167 [==============================] - 1s 944us/step - loss: 532.1461 - val_loss: 623.5695
Epoch 78/1000
1167/1167 [==============================] - 1s 987us/step - loss: 531.9331 - val_loss: 623.5082
Epoch 79/1000
1167/1167 [==============================] - 1s 955us/step - loss: 532.1595 - val_loss: 623.1230
Epoch 80/1000
1167/1167 [==============================] - 1s 869us/step - loss: 532.5941 - val_loss: 623.1398
Epoch 81/1000
1167/1167 [==============================] - 1s 892us/step - loss: 532.1995 - val_loss: 624.0635
Epoch 82/1000
1167/1167 [==============================] - 1s 939us/step - loss: 531.7440 - val_loss: 623.6414
Epoch 83/1000
1167/1167 [==============================] - 1s 1000us/step - loss: 532.3138 - val_loss: 623.9248
Epoch 84/1000


1167/1167 [==============================] - 1s 1ms/step - loss: 531.5668 - val_loss: 623.2090
Epoch 149/1000
1167/1167 [==============================] - 1s 1000us/step - loss: 531.3416 - val_loss: 623.7689
Epoch 150/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 531.2906 - val_loss: 623.8477
Epoch 151/1000
1167/1167 [==============================] - 1s 1ms/step - loss: 531.0729 - val_loss: 623.0307
Epoch 152/1000
1167/1167 [==============================] - 1s 954us/step - loss: 531.5522 - val_loss: 623.4417
Epoch 153/1000
1167/1167 [==============================] - 1s 966us/step - loss: 531.9876 - val_loss: 623.1292
Epoch 154/1000
1167/1167 [==============================] - 1s 992us/step - loss: 531.6371 - val_loss: 623.2031
Epoch 155/1000
1167/1167 [==============================] - 1s 925us/step - loss: 531.5746 - val_loss: 623.2747
Epoch 156/1000
1167/1167 [==============================] - 1s 906us/step - loss: 531.6602 - val_loss: 624.1758
Epoch 157/10

...

1150/1150 [==============================] - 1s 894us/step - loss: 533.7943 - val_loss: 441.5358
Epoch 63/1000
1150/1150 [==============================] - 1s 879us/step - loss: 533.7536 - val_loss: 440.1057
Epoch 64/1000
1150/1150 [==============================] - 1s 882us/step - loss: 533.8065 - val_loss: 440.9073
Epoch 65/1000
1150/1150 [==============================] - 1s 843us/step - loss: 534.4277 - val_loss: 441.7005
Epoch 66/1000
1150/1150 [==============================] - 1s 857us/step - loss: 534.1548 - val_loss: 440.8418
Epoch 67/1000
1150/1150 [==============================] - 1s 944us/step - loss: 534.4404 - val_loss: 440.5978
Epoch 68/1000
1150/1150 [==============================] - 1s 837us/step - loss: 534.2542 - val_loss: 440.8896
Epoch 69/1000
1150/1150 [==============================] - 1s 872us/step - loss: 534.0319 - val_loss: 441.3011
Epoch 70/1000
1150/1150 [==============================] - 1s 866us/step - loss: 533.6456 - val_loss: 440.9536
Epoch 71/1000
1

1150/1150 [==============================] - 1s 874us/step - loss: 532.7061 - val_loss: 439.9039
Epoch 137/1000
1150/1150 [==============================] - 1s 884us/step - loss: 532.9395 - val_loss: 440.5041
Epoch 138/1000
1150/1150 [==============================] - 1s 895us/step - loss: 533.1470 - val_loss: 441.1328
Epoch 139/1000
1150/1150 [==============================] - 1s 891us/step - loss: 532.8941 - val_loss: 440.7504
Epoch 140/1000
1150/1150 [==============================] - 1s 878us/step - loss: 532.7205 - val_loss: 440.3642
Epoch 141/1000
1150/1150 [==============================] - 1s 873us/step - loss: 532.4220 - val_loss: 441.2753
Epoch 142/1000
1150/1150 [==============================] - 1s 881us/step - loss: 532.6906 - val_loss: 440.4347
Epoch 143/1000
1150/1150 [==============================] - 1s 849us/step - loss: 532.3489 - val_loss: 440.2623
Epoch 144/1000
1150/1150 [==============================] - 1s 862us/step - loss: 532.9478 - val_loss: 441.7837
Epoch 1

1150/1150 [==============================] - 1s 1ms/step - loss: 532.0901 - val_loss: 440.8606
Epoch 210/1000
1150/1150 [==============================] - 1s 1ms/step - loss: 531.9355 - val_loss: 440.8694
Epoch 211/1000
1150/1150 [==============================] - 1s 998us/step - loss: 532.5192 - val_loss: 441.3836
Epoch 212/1000
1150/1150 [==============================] - 1s 1ms/step - loss: 532.5367 - val_loss: 440.9449
Epoch 213/1000
1150/1150 [==============================] - 1s 1ms/step - loss: 532.1072 - val_loss: 442.5970
Epoch 214/1000
1150/1150 [==============================] - 1s 907us/step - loss: 532.1610 - val_loss: 441.9713
Epoch 215/1000
1150/1150 [==============================] - 1s 936us/step - loss: 532.5897 - val_loss: 441.8523
Epoch 216/1000
1150/1150 [==============================] - 1s 924us/step - loss: 532.4617 - val_loss: 440.7011
Epoch 217/1000
1150/1150 [==============================] - 1s 989us/step - loss: 532.1723 - val_loss: 441.7937
Epoch 218/1000


1150/1150 [==============================] - 1s 961us/step - loss: 532.0229 - val_loss: 441.6676
Epoch 283/1000
1150/1150 [==============================] - 1s 963us/step - loss: 532.0917 - val_loss: 440.7376
Epoch 284/1000
1150/1150 [==============================] - 1s 927us/step - loss: 531.9025 - val_loss: 441.2947
Epoch 285/1000
1150/1150 [==============================] - 1s 974us/step - loss: 531.9144 - val_loss: 440.7130
Epoch 286/1000
1150/1150 [==============================] - 1s 938us/step - loss: 532.2270 - val_loss: 440.8433
Epoch 287/1000
1150/1150 [==============================] - 1s 1ms/step - loss: 531.9210 - val_loss: 440.9438
Epoch 288/1000
1150/1150 [==============================] - 1s 998us/step - loss: 531.9701 - val_loss: 440.9885
Epoch 289/1000
1150/1150 [==============================] - 1s 1ms/step - loss: 532.0169 - val_loss: 440.5089
Epoch 290/1000
1150/1150 [==============================] - 1s 961us/step - loss: 531.9642 - val_loss: 441.5999
Epoch 291/1

(25.11370326691211, 0.5795006584332028)

In [25]:
rmse_list

[16.65009880169072,
 19.98036995863874,
 13.91764028126171,
 17.484046821178662,
 26.68229436187206,
 29.440037308275517,
 20.764832241598498,
 17.691179365921325,
 41.53319507274803,
 24.00624406841042,
 26.388810328618654,
 17.6992044259869,
 22.185177616301566,
 45.72909401409058,
 18.897752769515797,
 23.570531278412595,
 26.122001031985228,
 19.928827439075945,
 17.792197830486618,
 16.83332629375624,
 36.270800154530626,
 17.929597108448,
 17.51318371273514,
 16.367637482495496,
 17.49710580225471,
 21.981710353216318,
 29.918318553418253,
 23.28441545358778,
 18.324540869301387,
 17.62073268648032,
 18.381039725374958,
 30.071146755152274,
 18.35583001598065,
 35.13949841983713]